# Fully Connected Naural Networks - Regularization - No Convolutions

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in __notMNIST_nonTensorFlow_comparisons.ipynb__.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## L2 regularization
L2 amounts to adding a penalty on the norm of the weights to the loss. 
In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). 

### Logistic Model 

In [5]:
image_size=28
##### logistic model 
def create_log_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         num_labels=10,batch_size = 128):
    
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+(1/batch_size)*beta*tf.nn.l2_loss(weights)

      # Optimizer.
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      
      valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
        
    test_accuracy = 0
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          test_accuracy = accuracy(test_prediction.eval(), test_labels)
          print("Test accuracy: %.1f%%" % test_accuracy)
    
    return test_accuracy
    

In [6]:
num_steps = 3001

betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_log_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i + 1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 17.438206
Minibatch accuracy: 9.4%
Validation accuracy: 12.8%
Test accuracy: 12.9%
Minibatch loss at step 500: 1.130953
Minibatch accuracy: 82.0%
Validation accuracy: 75.5%
Test accuracy: 83.2%
Minibatch loss at step 1000: 1.388350
Minibatch accuracy: 74.2%
Validation accuracy: 76.5%
Test accuracy: 83.8%
Minibatch loss at step 1500: 0.753861
Minibatch accuracy: 85.2%
Validation accuracy: 77.3%
Test accuracy: 84.6%
Minibatch loss at step 2000: 0.776861
Minibatch accuracy: 85.9%
Validation accuracy: 77.1%
Test accuracy: 85.0%
Minibatch loss at step 2500: 1.061428
Minibatch accuracy: 77.3%
Validation accuracy: 78.1%
Test accuracy: 85.4%
Minibatch loss at step 3000: 0.944870
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Test accuracy: 85.9%

>>>>>>>>>> Beta: 0.001000%
Initialized
Minibatch loss at step 0: 18.786240
Minibatch accuracy: 7.8%
Validation accuracy: 9.4%
Test accuracy: 9.2%
Minibatch loss at step 500: 1.354

In [7]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

*** Best beta:1 -- accuracy:88.71


We got an __improvement in test accuracy__ vs. not regularized model (~86.3%). 

### Neural Networks models: 1 hidden layer 

In [8]:
##### nn model 
import math

def create_nn1_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         hidden_size = 1024, 
                         num_labels=10,batch_size = 128):
    
    uniMax = 1/math.sqrt(hidden_size)
    
    with graph.as_default():
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.random_uniform([image_size * image_size, hidden_size], minval=-uniMax, maxval=uniMax),
                             name='weights_1')
      biases_1 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

      # Softmax 
      weights_2 = tf.Variable(tf.random_uniform([hidden_size, num_labels],minval=-uniMax, maxval=uniMax), name='weights_2')
      biases_2 = tf.Variable(tf.random_uniform([num_labels],minval=-uniMax, maxval=uniMax),name='biases_2')
      logits = tf.matmul(hidden_1, weights_2) + biases_2

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+(1/batch_size)*beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)) 

      # Optimizer
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
      test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
              print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [9]:
betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_nn1_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i +1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 2.306482
Minibatch accuracy: 12.5%
Validation accuracy: 39.6%
Test accuracy: 42.9%
Minibatch loss at step 500: 0.369548
Minibatch accuracy: 89.1%
Validation accuracy: 85.4%
Test accuracy: 91.8%
Minibatch loss at step 1000: 0.520196
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Test accuracy: 92.9%
Minibatch loss at step 1500: 0.309356
Minibatch accuracy: 92.2%
Validation accuracy: 87.6%
Test accuracy: 93.8%
Minibatch loss at step 2000: 0.270126
Minibatch accuracy: 93.0%
Validation accuracy: 87.8%
Test accuracy: 93.8%
Minibatch loss at step 2500: 0.335769
Minibatch accuracy: 89.1%
Validation accuracy: 88.3%
Test accuracy: 94.4%
Minibatch loss at step 3000: 0.346352
Minibatch accuracy: 89.1%
Validation accuracy: 88.5%
Test accuracy: 94.5%

>>>>>>>>>> Beta: 0.001000%
Initialized
Minibatch loss at step 0: 2.300950
Minibatch accuracy: 14.1%
Validation accuracy: 36.0%
Test accuracy: 39.4%
Minibatch loss at step 500: 0.3

In [10]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

*** Best beta:0.001 -- accuracy:94.51


We got an __improvement in test accuracy__ vs. not regularized model (~89.1%). 

### Neural Networks models: 2 hidden layers 

In [ ]:
def create_nn2_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         hidden_size = 1024, 
                         num_labels=10,batch_size = 128):
    
    uniMax = (1/math.sqrt(hidden_size))**2
    
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.random_uniform([image_size * image_size, hidden_size], minval=-uniMax, maxval=uniMax),
                             name='weights_1')
      biases_1 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
      # Hidden 2
      weights_2 = tf.Variable(tf.random_uniform([hidden_size, hidden_size], minval=-uniMax, maxval=uniMax),name='weights_2')
      biases_2 = tf.Variable(tf.random_uniform([hidden_size],minval=-uniMax, maxval=uniMax),name='biases_2')
      hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_2) + biases_2)

      # Softmax 
      weights_3 = tf.Variable(tf.random_uniform([hidden_size, num_labels],minval=-uniMax, maxval=uniMax), name='weights_3')
      biases_3 = tf.Variable(tf.random_uniform([num_labels],minval=-uniMax, maxval=uniMax),name='biases_3')
      logits = tf.matmul(hidden_2, weights_3) + biases_3

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )+(1/batch_size)*beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3))

      # Optimizer.
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2),
              weights_3)+biases_3)
      test_prediction = tf.nn.softmax(
            tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2),
              weights_3)+biases_3)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
    
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
              print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [ ]:
betas = [0, 0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_nn2_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i +1


>>>>>>>>>> Beta: 0.000000%
Initialized
Minibatch loss at step 0: 2.302630
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 500: 0.463727
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Test accuracy: 88.2%
Minibatch loss at step 1000: 0.519626
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Test accuracy: 91.5%
Minibatch loss at step 1500: 0.294864
Minibatch accuracy: 91.4%
Validation accuracy: 86.6%
Test accuracy: 93.3%
Minibatch loss at step 2000: 0.268220
Minibatch accuracy: 93.0%
Validation accuracy: 87.4%
Test accuracy: 94.0%


In [ ]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

We got an __improvement in test accuracy__ vs. not regularized model (~94.7%). 


## Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [ ]:
def create_nn_model_overfit_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         num_labels=10,batch_size = 128):
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024], name='weights_1'))
      biases_1 = tf.Variable(tf.zeros([1024]),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

      # Softmax 
      weights_2 = tf.Variable(tf.truncated_normal([1024, 10], name='weights_2'))
      biases_2 = tf.Variable(tf.zeros([10]),name='biases_2')
      logits = tf.matmul(hidden_1, weights_2) + biases_2

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+
                            beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)) )

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
      test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        init = tf.initialize_all_variables()
        session.run(init)
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (1 * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [ ]:
betas = [0.001,0.01,0.1,1,10]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_nn_model_overfit_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps)
   
  i = i +1

In [ ]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))

## Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
What happens to our extreme overfitting case?


In [ ]:
##### nn model with dropout 
def create_nn_model_dropout_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         dropout,
                         num_steps,
                         num_labels=10,batch_size = 128):
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024], name='weights_1'))
      biases_1 = tf.Variable(tf.zeros([1024]),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
      
      dropped = tf.nn.dropout(hidden_1, dropout)

      # Softmax 
      weights_2 = tf.Variable(tf.truncated_normal([1024, 10], name='weights_2'))
      biases_2 = tf.Variable(tf.zeros([10]),name='biases_2')
      logits = tf.matmul(dropped, weights_2) + biases_2

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
      test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        init = tf.initialize_all_variables()
        session.run(init)
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [ ]:
graph = tf.Graph()
test_accuracy = create_nn_model_dropout_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         0.5,
                         num_steps)
   

In [ ]:
def create_nn_model_overfit_dropout_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         dropout,
                         num_steps,
                         num_labels=10,batch_size = 128):
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024], name='weights_1'))
      biases_1 = tf.Variable(tf.zeros([1024]),name='biases_1')
      hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
      dropped = tf.nn.dropout(hidden_1, dropout)

      # Softmax 
      weights_2 = tf.Variable(tf.truncated_normal([1024, 10], name='weights_2'))
      biases_2 = tf.Variable(tf.zeros([10]),name='biases_2')
      logits = tf.matmul(dropped, weights_2) + biases_2

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
      test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        init = tf.initialize_all_variables()
        session.run(init)
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (1 * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [ ]:
graph = tf.Graph()
test_accuracy = create_nn_model_overfit_dropout_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         0.5,
                         num_steps)

## Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.
One avenue you can explore is to add multiple layers.
Another one is to use learning rate decay:
    

global_step = tf.Variable(0)  # count the number of steps taken.

learning_rate = tf.train.exponential_decay(0.5, global_step, ...)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [ ]:
##### nn model 

import math 

def create_multi_nn_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps,
                         num_labels=10,batch_size = 128,hidden_size=1024):
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Hidden 1
      #weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024], name='weights_1'))
      #biases_1 = tf.Variable(tf.zeros([1024]),name='biases_1')
      #hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
      # Hidden 2
      #weights_2 = tf.Variable(tf.truncated_normal([1024,1024], name='weights_2'))
      #biases_2 = tf.Variable(tf.zeros([1024]),name='biases_2')
      #hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_2) + biases_2)

      # Softmax 
      #weights_3 = tf.Variable(tf.truncated_normal([1024, 10], name='weights_3'))
      #biases_3 = tf.Variable(tf.zeros([10]),name='biases_3')
      #logits = tf.matmul(hidden_2, weights_3) + biases_3
        
        
      # Hidden 1
      with tf.name_scope('hidden1'):
        weights1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_size],
                                stddev=1.0 / math.sqrt(float(image_size * image_size))),
            name='weights1')
        biases1 = tf.Variable(tf.zeros([hidden_size]),
                             name='biases1')
        hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
        #dropped1 = tf.nn.dropout(hidden1, 0.5)
        
      # Hidden 2
      with tf.name_scope('hidden2'):
        weights2 = tf.Variable(
            tf.truncated_normal([hidden_size, hidden_size],
                                stddev=1.0 / math.sqrt(float(hidden_size))),
            name='weights2')
        biases2 = tf.Variable(tf.zeros([hidden_size]),
                             name='biases2')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
        #dropped2 = tf.nn.dropout(hidden2, 0.5)
        
      # Hidden 3
      #with tf.name_scope('hidden3'):
      #  weights3 = tf.Variable(
      #      tf.truncated_normal([1024, 1024],
      #                          stddev=1.0 / math.sqrt(float(1024))),
      #      name='weights2')
      #  biases3 = tf.Variable(tf.zeros([1024]),
      #                       name='biases_3')
      #  hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)
      #
        
      # Linear
      with tf.name_scope('softmax_linear'):
        weightsS = tf.Variable(
            tf.truncated_normal([hidden_size, num_labels],
                                stddev=1.0 / math.sqrt(float(hidden_size))),
            name='weightsS')
        biasesS = tf.Variable(tf.zeros([num_labels]),name='biasesS')
        logits = tf.matmul(hidden2, weightsS) + biasesS

      # 
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+
                            beta*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+
                                  tf.nn.l2_loss(weightsS)) )

      # Optimizer.
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      global_step = tf.Variable(0) # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

      # Predictions for the training, validation, and test data.     
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
                                       weights2) + biases2), weightsS) + biasesS)
      test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
                                       weights2) + biases2), weightsS) + biasesS)

    test_accuracy = 0
    with tf.Session(graph=graph) as session:
        init = tf.initialize_all_variables()
        session.run(init)
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0 or step ==(num_steps-1)):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
              test_accuracy = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy)
    return test_accuracy

In [ ]:
betas = [0,0.001,0.1,1]
test_accuracy = np.zeros(len(betas))
i = 0
for beta in betas:
  print("\n>>>>>>>>>> Beta: %f%%" % beta)
  graph = tf.Graph()
  test_accuracy[i] = create_multi_nn_model_and_run(graph,
                         train_dataset,
                         train_labels,
                         valid_dataset,
                         valid_labels,
                         test_dataset,
                         test_labels,
                         beta,
                         num_steps=3300,hidden_size=900)
   
  i = i +1

In [ ]:
print("*** Best beta:"+str(betas[np.argmax(test_accuracy)])+ " -- accuracy:" + str(test_accuracy[np.argmax(test_accuracy)]))